In [1]:
# default_exp organisms_data

# UniProt import for different organisms

## Dictionary linking organisms to their available fasta files and uniprot annotations

The import functions herein are only for loading data specified in the *all_organisms* dictionary. This means that the uniprot annotation files referred to herein have already been processed and formatted by alphamap. 

In [2]:
#export
all_organisms = {
    'Human': {'fasta_name': 'human.fasta',
               'uniprot_name': 'preprocessed_uniprot_human.csv'
              },
    'Mouse': {'fasta_name': 'mouse.fasta',
               'uniprot_name': 'preprocessed_uniprot_mouse.csv'
              },
    'Rat': {'fasta_name': 'rat_[10116].fasta',
               'uniprot_name': 'preprocessed_uniprot_rat.csv'
              },
    'Cow': {'fasta_name': 'bovine_[9913].fasta',
               'uniprot_name': 'preprocessed_uniprot_bovine.csv'
              },
    'Zebrafish': {'fasta_name': 'zebrafish_[7955].fasta',
               'uniprot_name': 'preprocessed_uniprot_zebrafish.csv'
              },
    'Drosophila': {'fasta_name': 'drosophila_[7227].fasta',
               'uniprot_name': 'preprocessed_uniprot_drosophila.csv'
              },
    'Caenorhabditis elegans': {'fasta_name': 'celegans_[6239].fasta',
               'uniprot_name': 'preprocessed_uniprot_human.csv'
              },
    'Slime mold': {'fasta_name': 'slimemold_[44689].fasta',
               'uniprot_name': 'preprocessed_uniprot_slimemold.csv'
              },
    'Arabidopsis thaliana': {'fasta_name': 'arabidopsis_thaliana.fasta',
               'uniprot_name': 'preprocessed_uniprot_arabidopsis.csv'
              },
    'Rice': {'fasta_name': 'rice_[39947].fasta',
               'uniprot_name': 'preprocessed_uniprot_rice.csv'
              },
    'Escherichia coli': {'fasta_name': 'ecoli_[83333].fasta',
               'uniprot_name': 'preprocessed_uniprot_ecoli.csv'
              },
    'Bacillus subtilis': {'fasta_name': 'bsubtilis_[224308].fasta',
               'uniprot_name': 'preprocessed_uniprot_bsubtilis.csv'
              },
    'Saccharomyces cerevisiae': {'fasta_name': 'yeast_[559292].fasta',
               'uniprot_name': 'preprocessed_uniprot_yeast.csv'
              },
    'SARS-CoV': {'fasta_name': 'cov.fasta',
               'uniprot_name': 'preprocessed_uniprot_cov.csv'
              },
    'SARS-CoV2': {'fasta_name': 'cov2.fasta',
               'uniprot_name': 'preprocessed_uniprot_cov2.csv'
              }
}

In [3]:
all_organisms['Rat']

{'fasta_name': 'rat_[10116].fasta',
 'uniprot_name': 'preprocessed_uniprot_rat.csv'}

## Function to load fasta file for a selected organism

In [4]:
#export
import os
import urllib.request
import shutil
import imp
from pyteomics import fasta
def import_fasta(organism: str):
    """
    Import fasta file for the selected organism.
    This downloads the file from github if not present.

    Args:
        organism (str): Organism for which the fasta file should be imported.
    Returns:
        fasta: Fasta file imported by pyteomics 'fasta.IndexedUniProt' for the selected organism.
    """
    if not organism in all_organisms.keys():
        raise ValueError(f"Organism {organism} is not available. Please select one of the following: {list(all_organisms.keys())}")


    BASE_PATH = imp.find_module("alphamap")[1] #os.path.abspath('')
    INI_PATH = os.path.join(BASE_PATH, '..')
    FUNCT_PATH = os.path.join(INI_PATH, 'alphamap')
    DATA_PATH = os.path.join(FUNCT_PATH, 'data')

    fasta_name = all_organisms[organism]['fasta_name']

    if not os.path.exists(os.path.join(DATA_PATH, fasta_name)):
        print(f"The fasta file for {organism} is downloaded from github.")
        github_url_data_folder = 'https://github.com/MannLabs/alphamap/blob/master/alphamap/data/'

        github_file = os.path.join(
            github_url_data_folder,
            os.path.basename(os.path.join(DATA_PATH, fasta_name))) + '/?raw=true'

        with urllib.request.urlopen(github_file) as response, open(os.path.join(DATA_PATH, fasta_name), 'wb') as out_file:
            shutil.copyfileobj(response, out_file)

    fasta_file = fasta.IndexedUniProt(os.path.join(DATA_PATH, fasta_name))

    return fasta_file

/var/folders/zx/n29r0swn0hddt1qgy159sjrw0000gn/T/ipykernel_17385/2091071375.py:5: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp


In [5]:
#hide

def test_import_fasta():
    
    # Test if exception works
    try:
        out = import_fasta(organism='rat')
    except ValueError as e:
        out = e
    assert str(out) == "Organism rat is not available. Please select one of the following: ['Human', 'Mouse', 'Rat', 'Cow', 'Zebrafish', 'Drosophila', 'Caenorhabditis elegans', 'Slime mold', 'Arabidopsis thaliana', 'Rice', 'Escherichia coli', 'Bacillus subtilis', 'Saccharomyces cerevisiae', 'SARS-CoV', 'SARS-CoV2']" 
    
    # Test if fasta is read correctly
    ecoli_fasta = import_fasta('Escherichia coli')
    assert ecoli_fasta[0].sequence == "MSQNTLKVHDLNEDAEFDENGVEVFDEKALVEQEPSDNDLAEEELLSQGATQRVLDATQLYLGEIGYSPLLTAEEEVYFARRALRGDVASRRRMIESNLRLVVKIARRYGNRGLALLDLIEEGNLGLIRAVEKFDPERGFRFSTYATWWIRQTIERAIMNQTRTIRLPIHIVKELNVYLRTARELSHKLDHEPSAEEIAEQLDKPVDDVSRMLRLNERITSVDTPLGGDSEKALLDILADEKENGPEDTTQDDDMKQSIVKWLFELNAKQREVLARRFGLLGYEAATLEDVGREIGLTRERVRQIQVEGLRRLREILQTQGLNIEALFRE"
    
test_import_fasta()

## Function to load uniprot annotations for a selected organism

In [6]:
#export
import os
import urllib.request
import shutil
import imp
import pandas as pd
def import_uniprot_annotation(organism: str):
    """
    Import uniprot annotation file for the selected organism.
    This downloads the file from github if not present.

    Args:
        organism (str): Organism for which the uniprot annotation should be imported.
    Returns:
        pd.DataFrame: Dataframe with the uniprot annotations for the selected organism.

    """
    if not organism in all_organisms.keys():
        raise ValueError(f"Organism {organism} is not available. Please select one of the following: {list(all_organisms.keys())}")


    BASE_PATH = imp.find_module("alphamap")[1] #os.path.abspath('')
    INI_PATH = os.path.join(BASE_PATH, '..')
    FUNCT_PATH = os.path.join(INI_PATH, 'alphamap')
    DATA_PATH = os.path.join(FUNCT_PATH, 'data')

    uniprot_name = all_organisms[organism]['uniprot_name']

    if not os.path.exists(os.path.join(DATA_PATH, uniprot_name)):
        print(f"The uniprot annotation file for {organism} is downloaded from github.")
        github_url_data_folder = 'https://github.com/MannLabs/alphamap/blob/master/alphamap/data/'

        github_file = os.path.join(
            github_url_data_folder,
            os.path.basename(os.path.join(DATA_PATH, uniprot_name))) + '/?raw=true'

        with urllib.request.urlopen(github_file) as response, open(os.path.join(DATA_PATH, uniprot_name), 'wb') as out_file:
            shutil.copyfileobj(response, out_file)

    uniprot_file = pd.read_csv(os.path.join(DATA_PATH, uniprot_name))

    return uniprot_file

In [7]:
#hide

def test_import_uniprot_annotation():
    
    # Test if exception works
    try:
        out = import_uniprot_annotation(organism='rat')
    except ValueError as e:
        out = e
    assert str(out) == "Organism rat is not available. Please select one of the following: ['Human', 'Mouse', 'Rat', 'Cow', 'Zebrafish', 'Drosophila', 'Caenorhabditis elegans', 'Slime mold', 'Arabidopsis thaliana', 'Rice', 'Escherichia coli', 'Bacillus subtilis', 'Saccharomyces cerevisiae', 'SARS-CoV', 'SARS-CoV2']" 
    
    # Test if fasta is read correctly
    ecoli_uniprot = import_uniprot_annotation('Escherichia coli')
    assert ecoli_uniprot.protein_id[0] == "P27685"
    
test_import_uniprot_annotation()

In [8]:
#hide

###### Export notebook to script ###### 

In [9]:
#hide
from nbdev.showdoc import *

ImportError: cannot import name 'AstRenderer' from 'mistune.renderers' (/opt/anaconda3/envs/alphamap/lib/python3.8/site-packages/mistune/renderers/__init__.py)

In [ ]:
#hide
from nbdev.export import *
notebook2script()